# Running on video

In [ ]:
import time
import cv2
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"]="1"

from Code.track import detect
from Code.get_prepared_data_sdd import prepared_data_sdd
from Code.visualize_sdd_annotation import visualizeSddAnnotation
from Code.extract_scene_seg import extract_scene_seg
from Code.preprocess import To_npz
import numpy as np

video_path = 'many_people.mp4'

#get the annotations from yolo and deepsort + resized images to 1920 x 1080
yolo_time_start = time.time()
dataset_resize,changelst , annotation = detect(video_path)
yolo_time_end = time.time()

#get box centre x,y for each person (traj_data)
#person_box_data : boxes coordinates for all persons
#other_box_data : boxes of other objects in the same frame with each targeted person
traj_data, person_box_data, other_box_data  = prepared_data_sdd(annotation,changelst)

#get the segmentation
seg_time_start = time.time()
model_path= 'deeplabv3_xception_ade20k_train/frozen_inference_graph.pb'
seg_output= extract_scene_seg(dataset_resize,model_path,every =2) #every: step by which it will skip frames 
seg_time_end = time.time()

#making npz which contanins arrays for details of the segmentation with annotations and person ids
data=To_npz(8,12,traj_data,seg_output)
np.savez("prepro_fold1/data_test.npz", **data)

simaug_time_start = time.time()
!python Code/test.py prepro_fold1/ packed_models/ best_simaug_model \
--wd 0.001 --runId 0 --obs_len 8 --pred_len 12 --emb_size 32 --enc_hidden_size 256 \
--dec_hidden_size 256 --activation_func tanh --keep_prob 1.0 --num_epochs 30 \
--batch_size 12 --init_lr 0.3 --use_gnn --learning_rate_decay 0.95 --num_epoch_per_decay 5.0 \
--grid_loss_weight 1.0 --grid_reg_loss_weight 0.5 --save_period 3000 \
--scene_h 36 --scene_w 64 --scene_conv_kernel 3 --scene_conv_dim 64 \
--scene_grid_strides 2,4 --use_grids 1,0 --val_grid_num 0 --gpuid 0 --load_best \
--save_output sdd_out.p
simaug_time_end = time.time()


In [ ]:
print(f"yolo time : {yolo_time_end-yolo_time_start}")
print(f"seg time : {seg_time_end-seg_time_start}")
print(f"simaug time : {simaug_time_end-simaug_time_start}")
print(f"whole time : {simaug_time_end-yolo_time_start}")

In [ ]:
#get the frames to visualize on it 
resized_videos_frames_path= r"resized_videos_frames"
for i,j in dataset_resize.items():
    cv2.imwrite(os.path.join(resized_videos_frames_path, "%s_F_%08d.jpg" % ("stream", i)), j)

In [ ]:
#get the frames with detection for one person 
person_id = 1
frames_with_detection = visualizeSddAnnotation(dataset_resize, changelst, traj_data, person_box_data, other_box_data,person_id)
frames_with_detection_path= r"frames_with_detection"
for i,j in frames_with_detection.items():
    plt.imshow(j)
    plt.savefig(os.path.join(frames_with_detection_path, "%s.jpg" % ( i)), dpi=100)
    plt.clf()

In [ ]:
#get the heatmap frames
!echo fsdd_out_0.p,0_0_255 > sdd_run.lst
!python code/visualize_output.py sdd_run.lst "resized_videos_frames" "heatmap/" \
--vis_num 50   --use_heatmap --ordered

In [ ]:
#get the segmented frames
segmented_path= r"segmented"
for i,j in seg_output.items():
    plt.imshow(j)
    plt.savefig(os.path.join(segmented_path, "%s.jpg" % ( i)), dpi=100)
    plt.clf()

# Running on stream

In [ ]:
import time
from Code.track_stream import detect
from Code.queue import get_queue
from Code.get_prepared_data_sdd import prepared_data_sdd
from Code.visualize_sdd_annotation import visualizeSddAnnotation
from Code.extract_scene_seg import extract_scene_seg
from Code.preprocess import To_npz
import cv2
import numpy as np
import os
# cv2.namedWindow("webcam", cv2.WINDOW_NORMAL)
camera = cv2.VideoCapture(0)
for i in range(1):
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
    os.environ["NUMEXPR_NUM_THREADS"] = "1"
    os.environ["KMP_DUPLICATE_LIB_OK"]="1"
    images_try = get_queue(5,12,camera)
    dataset_resize,changelst , annotation = detect(images_try)
    traj_data, person_box_data, other_box_data = prepared_data_sdd(annotation,changelst)
    # framesData = visualizeSddAnnotation(dataset_resize, changelst, traj_data, person_box_data, other_box_data)
    model_path= 'deeplabv3_xception_ade20k_train/frozen_inference_graph.pb'
    seg_output= extract_scene_seg(dataset_resize,model_path,every =2)
    data=To_npz(4,6,traj_data,seg_output)
    np.savez("prepro_fold1/data_test.npz", **data)
    !python Code/test.py prepro_fold1/ packed_models/ best_simaug_model \
    --wd 0.001 --runId 0 --obs_len 4 --pred_len 6 --emb_size 32 --enc_hidden_size 256 \
    --dec_hidden_size 256 --activation_func tanh --keep_prob 1.0 --num_epochs 30 \
    --batch_size 12 --init_lr 0.3 --use_gnn --learning_rate_decay 0.95 --num_epoch_per_decay 5.0 \
    --grid_loss_weight 1.0 --grid_reg_loss_weight 0.5 --save_period 3000 \
    --scene_h 36 --scene_w 64 --scene_conv_kernel 3 --scene_conv_dim 64 \
    --scene_grid_strides 2,4 --use_grids 1,0 --val_grid_num 0 --gpuid 0 --load_best \
    --save_output "sdd_out_{i}.p"

In [ ]:
camera.release()